In this example, we want to explain how Functional Graphical Lasso works and how you can make use of it with GGLasso. For this tutorial, we use the scikit-fda package.

For Functional Graphical Lasso every variable is representing a function or time series. In order to obtain a finite-dimensional problem, we represent the function in some basis (e.g. by computing Fourier coefficients and truncating). Then, we compute correlations of the corresponding coefficients and are interested in the relationships between different functional variables.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from skfda.datasets import make_gaussian_process
from skfda.misc.covariances import *
from skfda.misc.metrics import l2_distance, l2_norm
from skfda.representation.basis import BSpline, Fourier
from skfda.representation.grid import FDataGrid

We work in the following setup: we create (functional) variables, called $v_0, v_1, \dots$. Some of those will come from a Gaussian process, some will be constructed in particular ways. We then draw samples for each variable.

In the next step, we will compute representations with respect to some basis. Here, we have the option between Fourier basis and BSplines.

In [ ]:
t0 = 0
t1 = 4

n_ts = 101
n_samples = 10**2

_t = np.linspace(t0,t1,n_ts)

def get_basis(style, n_components):
    if style == 'bspline':
        basis = BSpline(domain_range=(t0, t1), n_basis=n_components, order=4)
    elif style == 'fourier':
        basis = Fourier(domain_range=(t0, t1), n_basis=n_components, period=None)
    return basis

# decide on a basis
style = 'fourier'

In [ ]:
# generate samples from a simple function
def hat(t,A): 
    y = (t<= t1/2) * (2*A/t1 * t) + (t> t1/2) *(2*A - 2*A/t1 * t)
    return y

modes = 4*np.random.rand(n_samples) - 2
v2 = np.zeros((n_samples, n_ts))

for j in range(n_samples):
    v2[j,:] = hat(_t, modes[j])

In [ ]:
# define three variables
cov0 = Gaussian(variance=1., length_scale=1.)
cov1 = Gaussian(variance=1, length_scale=0.5)
cov2 = Linear(variance=0.1) #not used 

all_var = [cov0, cov1, v2]
n_var = len(all_var)

In [ ]:
# plot covariance functions
_ = cov0.heatmap()
_ = cov1.heatmap()
_ = cov2.heatmap()

In [ ]:
# sample time series
samples = list()
labels = list()

for j in np.arange(n_var):
    #_ds.data_matrix has shape (n_ts, n_samples, 1)
    if not isinstance(all_var[j], np.ndarray):
        _ds = make_gaussian_process(
                n_samples=n_samples,
                n_features=n_ts,
                start=t0,
                stop=t1,
                cov=all_var[j],
                mean=0,
                random_state=20
                )
    else:
        _ds = FDataGrid(all_var[j], _t)
    
    _lb = np.array([f'v{j}'] * n_samples)
    
    samples.append(_ds)
    labels.append(_lb)


We can now plot the sampled time series for each variable. The first two come from a Gaussian process where the respective kernels have different length scales. The third one is simply a linearly increasing and the decreasing function.

In [ ]:
# plot sampled time series
colors = ['darkred', 'C1', 'grey']
_alpha = 0.2
_lw = 2.

fig, axs = plt.subplots(n_var,1, figsize=(10,12))

for j in range(n_var):
    ax = axs.ravel()[j]
    samples[j].plot(axes=ax, group=labels[j], group_colors={f'v{j}': colors[j]}, alpha=_alpha, lw=_lw)
    ax.set_title(f'Sampled time series for v{j}')

In the next step, we compute a basis representation (for a finite number of basis components!), reconstruct the time series and plot the reconstruction as well as the error.

In [ ]:
# do basis transformation, reconstruct and plot deviation
q = 7
fig, axs = plt.subplots(n_var, 2, figsize=(15,9))

for j in range(n_var):
    basis = get_basis(style, q)
    traf = samples[j].to_basis(basis)

    recov_ds = traf.to_grid(samples[j].grid_points)
    diff = samples[j] - recov_ds
        
    ax = axs[j,0]
    recov_ds.plot(axes=ax, group=labels[j], group_colors={f'v{j}': colors[j]}, alpha=_alpha, lw=_lw)
    ax.set_title(f'Basis representation - reconstructed time series for v{j}')
    
    ax2 = axs[j,1]
    diff.plot(axes=ax2, group=labels[j], group_colors={f'v{j}': colors[j]}, alpha=_alpha, lw=_lw)
    ax2.set_title(f'Reconstruction error for v{j}')
    ax2.set_ylim(ax.get_ylim())
    
fig.tight_layout()

## Fourth variable construction

We want to create a fourth variable that is constructed using the first variable $v_0$ as follows: we permute the coefficients of the basis represenation and then retransform to a time series, but with the basis elements not permuted. This creates a variable where the relationship to the original $v_0$ is not obvious if only looking at the time series.

In [ ]:
# components for v3 
n_comp = 9

In [ ]:
basis = get_basis(style, n_comp)
_traf = samples[0].to_basis(basis)   

rng = np.random.default_rng(1917)
_perm = rng.permutation(n_comp)

# permute
_traf.coefficients = _traf.coefficients[:, _perm]

# retransform to time series
v3_ds = _traf.to_grid(samples[0].grid_points)

# add some noise
# v3_ds.data_matrix +=  0.05*np.random.randn(n_samples, n_ts, 1)

In [ ]:
n_var += 1

# add new time series
samples.append(v3_ds)
_lb3 = np.array([f'v3'] * n_samples)
labels.append(_lb3)
colors.append('steelblue')

# plot again
fig, axs = plt.subplots(n_var,1, figsize=(10,12))
for j in range(n_var):
    ax = axs.ravel()[j]
    samples[j].plot(axes=ax, group=labels[j], group_colors={f'v{j}': colors[j]}, alpha=_alpha, lw=_lw)
    ax.set_title(f'Sampled time series for v{j}')

fig.tight_layout()

In [ ]:
# plot basis functions
basis = get_basis(style, n_comp)
_x = np.linspace(t0,t1,100)
W = basis.evaluate(_x).squeeze()

fig, ax = plt.subplots(figsize=(10,5))
for i in range(n_comp):
    ax.plot(_x, W[i,:], lw=2, label=f"basis {i}")

ax.legend()
fig.suptitle("Plot of the basis functions")

## Reconstruction error

Now, we compute the recosntruction error (as a relative median l2 error) for an increasing number of basis components. We would expect that the error goes to zero.

In [ ]:
fig, ax = plt.subplots()
    
for j in range(n_var):
    all_err = list()
    
    if style != 'fourier':
        all_q = range(4,12)
    else:
        all_q = [5,7,9,11,13] # Fourier has always odd number of basis elements
        
    for k in range(len(all_q)):
        basis = get_basis(style, all_q[k])
        this_traf = samples[j].to_basis(basis)
        
        # reconstruct ds 
        recov_ds = this_traf.to_grid(samples[j].grid_points)
    
        # compute error
        this_err = l2_distance(samples[j], recov_ds) / l2_norm(samples[j])
        all_err.append(np.median(this_err))

   
    ax.plot(all_q, all_err, c=colors[j], lw = 4, marker='p', markersize=10, markeredgecolor='k', alpha=0.8, label=f'v{j}')
    
ax.set_xlabel('Number of basis components')
ax.set_ylabel('Median l2 error')
ax.grid(ls = '-', lw = .5) 
ax.legend()

## Functional Graphical Lasso (FSGL)

As seen above, with sufficiently man basis components we have a good representation of the original function.
Now, compute the coefficients for the first $n_{comp}$ basis components. Then, compute their correlations which will be the input for Functional Graphical Lasso.

If we choose $n_{comp}=9$, we expect $4\cdot9=36$ variables in total.

In [ ]:
# from now on use 9 components
n_comp = 9

In [ ]:
from gglasso.helper.basic_linalg import scale_array_by_diagonal
from gglasso.helper.utils import lambda_max_fsgl, frob_norm_per_block
from gglasso.solver.functional_sgl_admm import ADMM_FSGL
from gglasso.helper.experiment_helper import plot_fsgl_heatmap

p = n_var
M = n_comp
pM = p*M
all_traf = list()
    
# compute coefficients for each variable
for j in range(n_var):
    basis = get_basis(style, n_comp)
    print(basis)
    _traf = samples[j].to_basis(basis)
    
    all_traf.append(_traf.coefficients)
    
Z = np.hstack(all_traf)
print("(N,p) = ", Z.shape)

# compute covariances
S = np.cov(Z.T)
# scale to correlations
S = scale_array_by_diagonal(S)

print("S has shape ", S.shape)

In [ ]:
# plot samples
fig, ax = plt.subplots(figsize=(15,10))
sns.heatmap(Z, ax=ax, alpha=1, vmin=-3, vmax=3, cmap='coolwarm', cbar=True, linewidth = 0.005, linecolor = 'lightgrey',)
ax.vlines([(j+1)*n_comp for j in range(n_var)], 0, n_samples, color='k', lw=4)
ax.set_title('Basis coefficients for all samples and variables')

# plot heatmap
fig, axs = plt.subplots(1,2,figsize=(9,4), gridspec_kw={'width_ratios': [5.5,4.5]})
_v = 1.
ax = axs[0]
plot_fsgl_heatmap(S, p, M, ax=ax)
ax.set_title('Empirical correlation matrix')

# plot Frobenius norm of each subblock
ax = axs[1]
to_plot = np.round(frob_norm_per_block(S, n_comp),3)
sns.heatmap(to_plot, annot=True, vmin=0, vmax=to_plot.max(), linewidth=0.005, linecolor='w', cbar=False, ax=ax)
ax.set_title(r"Heatmap of $\||\Theta^M_{jl}\||_F$")

For FSGL, we have to specify a regularization parameter $\lambda$. Typically, one computes the solution for a range of $\lambda$ values and then chooses the best suited solution according to some criterion.

In [ ]:
# create range of lambda values
lambda_max = lambda_max_fsgl(S, M)
lambda_min = 0.1 * lambda_max
lambda_range = np.logspace(np.log10(lambda_min), np.log10(lambda_max), 9)[::-1]

In [ ]:
lambda_range

In [ ]:
# solve Functional Graphical Lasso for all lambda
Omega_0 = np.eye(pM)
all_sol = dict()

for j in range(len(lambda_range)):
    _lam = lambda_range[j]
    sol, info = ADMM_FSGL(S, _lam, M, Omega_0,
                         rho=1., max_iter=2000, tol=1e-8, rtol=1e-7,\
                         update_rho=True, verbose=False, measure=True, latent=False, mu1=None)
    
    Omega_0 = sol['Omega'].copy() # warm start
    all_sol[_lam] = sol.copy()

As $v_3$ was constructed from $v_0$, we would expect that their relationship can be recovered. Also, $v_0$ and $v_1$ are more related as $v_0$ and $v_2$ as both come from a Gaussian process whereas $v_2$ was piecewise linear.
Let's see whether these relationships are correctly identified by FSGL. For this, we are only interested whether the corresponding block (of size MxM) is **non-zero**. The block itself could be sparse or dense and its individual entries are harder to interpret.

In [ ]:
fig, axs = plt.subplots(3,3, figsize=(17,15))

for j in range(len(lambda_range)):
    _lam = lambda_range[j]
    ax = axs.ravel()[j]
    plot_fsgl_heatmap(all_sol[_lam]['Theta'], p, M, ax=ax)
    ax.set_title(fr"$\lambda$={_lam}")
    
fig.suptitle(r'FSGL solution for all $\lambda$-values')

Finally, we plot the Frobenius norm of each subblock for each $\lambda$-value. Like this, we can see which block first (i.e. for the largest $\lambda$) enters the solution. As expected it is the block encoding the relationship between $v_0$ and $v_3$.

In [ ]:
Fnorm = np.zeros((len(lambda_range), p, p))

for j in range(len(lambda_range)):
    _lam = lambda_range[j]
    Fnorm[j,:,:] = frob_norm_per_block(all_sol[_lam]['Theta'], M)

# set 0 entries to nan for plotting
Fnorm[Fnorm <= 1e-8] = np.nan

In [ ]:
fig, ax = plt.subplots()

for i in range(p):
    for j in np.arange(start=i+1, stop=p):
        ax.plot(lambda_range, Fnorm[:,i,j], lw=3, marker='o', markersize=8, alpha=0.7, markevery=(1,i+1),
                markeredgecolor='k',
                label = f'block {i}_{j}')

ax.set_xscale('log')
ax.set_ylim(0,)
ax.set_xlim(lambda_range[-1], lambda_range[0])

ax.set_xlabel(r'$\lambda$', fontsize=14)
ax.set_ylabel(r"$\||\Theta^M_{jl}\||_F$", fontsize=14)
ax.grid(ls = '-', lw = .5) 
ax.legend()